In [10]:
from langchain.llms import VertexAI

# LLM model
llm = VertexAI(
    model_name="text-bison-32k",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI(
    model_name = "chat-bison-32k",
)

In [1]:
from langchain.chat_models import ChatVertexAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

In [2]:
chat=ChatVertexAI()

cloudpickle is not installed. Please call `pip install google-cloud-aiplatform[preview]`.


In [3]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
chat(messages)

AIMessage(content=" J'aime la programmation.", additional_kwargs={}, example=False)

# Prompt Template

In [4]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="gruesome", content="goats")

'Tell me a gruesome joke about goats.'

# Chat Template

In [6]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

ValidationError: 8 validation errors for ChatPromptTemplate
messages -> 0
  value is not a valid dict (type=type_error.dict)
messages -> 0
  value is not a valid dict (type=type_error.dict)
messages -> 1
  value is not a valid dict (type=type_error.dict)
messages -> 1
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 3
  value is not a valid dict (type=type_error.dict)
messages -> 3
  value is not a valid dict (type=type_error.dict)

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatVertexAI

llm = ChatVertexAI()
llm(template.format_messages(text='i dont like eating british food, and its weather is very depressing'))

AIMessage(content=" While British food may not be to everyone's taste, there are many delicious dishes to be found in the UK. From classic fish and chips to hearty stews and pies, there's something for everyone to enjoy. And while the weather in the UK can be unpredictable, there are still plenty of beautiful days to be enjoyed. Why not take a walk in the countryside or explore one of the many museums or art galleries?", additional_kwargs={}, example=False)

# Creating a Chain

In [14]:
import os
from getpass import getpass
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {topic}"
)

prompt_template.format(adjective="offensive", topic="indian people")

'Tell me a offensive joke about indian people'

#### Callbacks (not quite sure I understand them yet)

In [15]:
# token counting
import asyncio

from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
with get_openai_callback() as cb:
    llm("What is the square root of 4?")

total_tokens = cb.total_tokens
assert total_tokens > 0

with get_openai_callback() as cb:
    llm("What is the square root of 4?")
    llm("What is the square root of 4?")

assert cb.total_tokens == total_tokens * 2

# You can kick off concurrent runs from within the context manager
with get_openai_callback() as cb:
    await asyncio.gather(
        *[llm.agenerate(["What is the square root of 4?"]) for _ in range(3)]
    )

assert cb.total_tokens == total_tokens * 3

# The context manager is concurrency safe
task = asyncio.create_task(llm.agenerate(["What is the square root of 4?"]))
with get_openai_callback() as cb:
    await llm.agenerate(["What is the square root of 4?"])

await task
assert cb.total_tokens == total_tokens

In [16]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
prompt = PromptTemplate.from_template("1 + {number} = ")

# Constructor callback: First, let's explicitly set the StdOutCallbackHandler when initializing our chain
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=2)

# Use verbose flag: Then, let's use the `verbose` flag to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.run(number=2)

# Request callbacks: Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=2, callbacks=[handler])



> Entering new  chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new  chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new  chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


'\n\n3'

# Retrival

In [17]:
# Document loaders
from langchain.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()

[Document(page_content='hello\n\n# What is your name?\n', metadata={'source': './index.md'})]

# Web based loader

In [18]:

from langchain.document_loaders import WebBaseLoader
     

loader = WebBaseLoader("https://cloud.google.com/blog/products/ai-machine-learning/generative-ai-applications-with-vertex-ai-palm-2-models-and-langchain")     
data = loader.load()

In [19]:
print(f"Found {len(data)} comments")
print(f"Here's a sample:\n\n{''.join([x.page_content[:500] for x in data[:2]])}")

Found 1 comments
Here's a sample:

Generative AI applications with Vertex AI PaLM 2 Models and LangChain | Google Cloud BlogJump to ContentCloudBlogContact sales Get started for free CloudBlogSolutions & technologyAI & Machine LearningAPI ManagementApplication DevelopmentApplication ModernizationChrome EnterpriseComputeContainers & KubernetesData AnalyticsDatabasesDevOps & SREMaps & GeospatialSecurity & IdentityInfrastructureInfrastructure ModernizationNetworkingProductivity & CollaborationSAP on Google CloudStorage & Data Transf


# Memory

In [20]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")
     

conversation.predict(input="What is the capital of Spain?")
     

conversation.predict(input="What are some popular places I can see in France?")
     

conversation.predict(input="What question did I ask first?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: What is the capital of Spain?
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI 

' You asked what the capital of Spain is.'

# Chains

### Sequential Chain

In [21]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [22]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [23]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [27]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

review = overall_chain.run("Madrid")



> Entering new  chain...
A classic dish from Madrid is Cocido Madrileño, a hearty stew made with chickpeas, vegetables, and various cuts of meat.

Cocido Madrileño Recipe

Ingredients:
- 2 cups dried chickpeas, soaked overnight
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 carrots, diced
- 2 celery stalks, diced
- 2 potatoes, diced
- 2 cups chicken broth
- 1/2 pound chorizo sausage, cut into 1/2-inch slices
- 1/2 pound smoked ham, cut into 1/2-inch cubes
- 1/2 pound beef chuck, cut into 1/2-inch cubes
- 1/2 pound pork shoulder, cut into 1/2-inch cubes
- 1/2 cup white wine
- 2 bay leaves
- Salt and pepper to taste

Instructions:
1. Heat the olive oil in a large pot over medium heat. Add the onion, garlic, carrots, and celery and cook until softened, about 5 minutes.
2. Add the potatoes, chicken broth, chorizo, ham, beef, pork, and soaked chickpeas. Bring to a boil, then reduce heat to low and simmer for 1 hour.
3. Add the white wine, bay leaves, salt, and

>

### Summarisation Chain

In [25]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    "https://cloud.google.com/blog/products/ai-machine-learning/how-to-use-grounding-for-your-llms-with-text-embeddings"
)
documents = loader.load()

print(f"# of words in the document = {len(documents[0].page_content)}")

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# There is a lot of complexity hidden in this one line. I encourage you to check out the video above for more detail
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)

# of words in the document = 13621


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"How to use Grounding for your LLMs with text embeddings | Google Cloud BlogJump to ContentCloudBlogContact sales Get started for free CloudBlogSolutions & technologyAI & Machine LearningAPI ManagementApplication DevelopmentApplication ModernizationChrome EnterpriseComputeContainers & KubernetesData AnalyticsDatabasesDevOps & SREMaps & GeospatialSecurity & IdentityInfrastructureInfrastructure ModernizationNetworkingProductivity & CollaborationSAP on Google CloudStorage & Data TransferSustainabilityEcosystemIT LeadersIndustriesFinancial ServicesHealthcare & Life SciencesManufacturingMedia & EntertainmentPublic SectorRetailSupply ChainTelecommunicationsPartnersStartups & SMBTraining & CertificationsInside Google CloudGoogle Cloud Next & EventsGoogle Maps PlatformGoogle WorkspaceDevelopers & PractitionersTransform with Google CloudCo

" This article explains how startups, AI, and machine learning are being used to improve customer experiences, automate document processing, and help farmers. It covers the concept of grounding with embeddings and vector search to build reliable AI services for enterprise use, and introduces LangChain and Vertex AI as popular tools for grounding LLMs. It also discusses Google Cloud Vertex AI Model Garden, which includes Embeddings for Text and Image APIs, and Nomic AI's Embeddings API and Atlas platform. Finally, it covers the use of Google Research's ScaNN algorithm and Vertex AI Matching Engine for vector search."